<a href="https://colab.research.google.com/github/asternoeld/introduction-to-python/blob/main/kiva_loans_analysis_aster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kiva Loans Analysis  
### Assignment 3 – Pandas

**Name:** Aster Noel Dsouza  
**Student ID:** 29211  
**Date:** NOVEMBER 18, 2025



## Introduction

In this notebook I analyse the **Kiva loans** dataset (`kiva_loans.csv`) using the Python library **pandas**.  
Kiva is a crowdfunding platform where lenders fund loans for borrowers around the world. The dataset contains information about loan amounts, sectors, countries, borrower genders, currencies, dates, and other details.

The goal of this assignment is to:
- Practice working with pandas DataFrames.
- Explore the dataset by asking and answering **8 questions**.
- Use important operations such as:
  - Filtering rows with string methods and numerical conditions.
  - Sorting tables.
  - Grouping and aggregating data with `groupby`.
  - Creating new variables using `apply` and `lambda`.

In the video presentation I will focus on 6 of these questions, but all 8 questions are included here for practice.


## 0. Setup and data loading

In this section I will:

- Import the libraries I need (mainly `pandas`).
- Automatically download the Kiva dataset from Kaggle using `kagglehub`.
- Load the `kiva_loans.csv` file into a DataFrame.
- Take a quick look at the structure of the dataset (columns, data types, and a few example rows).


In [3]:
# 0. Setup and data loading

# Install kagglehub (only needs to run once per session)
!pip install kagglehub --quiet

import os
import pandas as pd
import kagglehub

# Download latest version of the Kiva dataset from Kaggle
path = kagglehub.dataset_download("kiva/data-science-for-good-kiva-crowdfunding")
print("Path to dataset files:", path)

# Build the full path to the main CSV file
csv_path = os.path.join(path, "kiva_loans.csv")

# Load the CSV into a pandas DataFrame
df = pd.read_csv(csv_path)

# Quick checks on the data
print("Number of rows and columns:", df.shape)
print("\nColumn names:")
print(df.columns)

# Look at the first 5 rows
df.head()


Using Colab cache for faster access to the 'data-science-for-good-kiva-crowdfunding' dataset.
Path to dataset files: /kaggle/input/data-science-for-good-kiva-crowdfunding
Number of rows and columns: (671205, 20)

Column names:
Index(['id', 'funded_amount', 'loan_amount', 'activity', 'sector', 'use',
       'country_code', 'country', 'region', 'currency', 'partner_id',
       'posted_time', 'disbursed_time', 'funded_time', 'term_in_months',
       'lender_count', 'tags', 'borrower_genders', 'repayment_interval',
       'date'],
      dtype='object')


,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
0,653051,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12.0,12,NaN,female,irregular,2014-01-01
1,653053,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11.0,14,NaN,"female, female",irregular,2014-01-01
2,653068,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,334.0,2014-01-01 09:58:07+00:00,2013-12-17 08:00:00+00:00,2014-01-01 16:01:36+00:00,43.0,6,"user_favorite, user_favorite",female,bullet,2014-01-01
3,653063,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 08:03:11+00:00,2013-12-24 08:00:00+00:00,2014-01-01 13:00:00+00:00,11.0,8,NaN,female,irregular,2014-01-01
4,653084,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,245.0,2014-01-01 11:53:19+00:00,2013-12-17 08:00:00+00:00,2014-01-01 19:18:51+00:00,14.0,16,NaN,female,monthly,2014-01-01


## Table of Contents

1. [Question 1 – Which sectors and activities are most common?](#Question-1-–-Which-sectors-and-activities-are-most-common)
2. [Question 2 – For a specific currency, which sectors receive the largest loans?](#Question-2-–-For-a-specific-currency,-which-sectors-receive-the-largest-loans)
3. [Question 3 – Do female-only, male-only, and mixed-gender groups receive different loan sizes?](#Question-3-–-Do-female-only,-male-only,-and-mixed-gender-groups-receive-different-loan-sizes)
4. [Question 4 – How do selected countries compare in loan amounts and lender counts?](#Question-4-–-How-do-selected-countries-compare-in-loan-amounts-and-lender-counts)
5. [Question 5 – Is there a relationship between the number of lenders and the loan amount?](#Question-5-–-Is-there-a-relationship-between-the-number-of-lenders-and-the-loan-amount)
6. [Question 6 – Do shorter-term loans get funded faster than longer-term loans?](#Question-6-–-Do-shorter-term-loans-get-funded-faster-than-longer-term-loans)
7. [Question 7 – How many borrowers are in each loan, and does this vary by sector or country?](#Question-7-–-How-many-borrowers-are-in-each-loan,-and-does-this-vary-by-sector-or-country)
8. [Question 8 – Are loans with certain tags different in size from the overall average?](#Question-8-–-Are-loans-with-certain-tags-different-in-size-from-the-overall-average)


## Question 1 – Which sectors and activities are most common?

In this section I want to get a first idea of what kind of loans appear in the Kiva dataset.

I will:
- Look at the most frequent **sectors** of loans.
- Also check the most common **activities**.
- Compute, for each sector, how many loans there are and the average loan amount.
- Take into account that loan amounts are recorded in different **currencies**, so I will keep the currency information in the analysis (for example, by grouping by sector *and* currency).

This will help me understand which areas Kiva focuses on the most.


In [4]:
# 1) Most common sectors

sector_counts = (
    df["sector"]
    .value_counts()
    .reset_index()
)

sector_counts.columns = ["sector", "loan_count"]

print("Top 10 sectors by number of loans:")
display(sector_counts.head(10))

# 2) Most common activities

activity_counts = (
    df["activity"]
    .value_counts()
    .reset_index()
)

activity_counts.columns = ["activity", "loan_count"]

print("Top 10 activities by number of loans:")
display(activity_counts.head(10))


Top 10 sectors by number of loans:


,sector,loan_count
0,Agriculture,180302
1,Food,136657
2,Retail,124494
3,Services,45140
4,Personal Use,36385
5,Housing,33731
6,Clothing,32742
7,Education,31013
8,Transportation,15518
9,Arts,12060


Top 10 activities by number of loans:


,activity,loan_count
0,Farming,72955
1,General Store,64729
2,Personal Housing Expenses,32448
3,Food Production/Sales,28106
4,Agriculture,27023
5,Pigs,26624
6,Retail,24771
7,Clothing Sales,22339
8,Home Appliances,20267
9,Higher education costs,19742


In [ ]:
# Group by sector and currency
sector_currency_summary = (
    df.groupby(["sector", "currency"], as_index=False)
      .agg(
          loan_count=("id", "count"),               # how many loans
          avg_loan_amount=("loan_amount", "mean")   # average loan amount
      )
)

# Sort by number of loans (descending)
sector_currency_sorted = sector_currency_summary.sort_values(
    by="loan_count",
    ascending=False
)

print("Top 15 sector+currency combinations by number of loans:")
display(sector_currency_sorted.head(15))


**Summary for Question 1**

From the tables above I can see which sectors and activities appear most often in the Kiva loans dataset.  
Also, by grouping by sector and currency, I can see which combinations are the most common and what the typical loan amount is for each one. This gives a first overview of the main areas where Kiva operates.


## Question 2 – For a specific currency, which sectors receive the largest loans?

Because loan amounts use different currencies, it is hard to compare them all at once.  
In this question I will focus on **one single currency** (for example `USD` or another currency with many loans) and compare sectors only within that currency.

I will:
- Filter the dataset to keep only loans in the chosen currency.
- Optionally remove loans with missing or zero loan amounts.
- Group the data by sector and calculate:
  - The number of loans per sector.
  - The average loan amount per sector.
- Sort the results to see which sectors receive the biggest loans on average in that currency.

This gives a clearer and fair comparison of loan sizes across sectors.


In [5]:
# Quick look at the most common currencies
currency_counts = df["currency"].value_counts().reset_index()
currency_counts.columns = ["currency", "loan_count"]

print("Top 10 currencies by number of loans:")
display(currency_counts.head(10))


Top 10 currencies by number of loans:


,currency,loan_count
0,PHP,160440
1,USD,105494
2,KES,75311
3,KHR,29498
4,PKR,26856
5,COP,21984
6,PEN,21401
7,UGX,20534
8,XOF,18610
9,TJS,17553


In [6]:
# Choose one currency to focus on
chosen_currency = "USD"  # <-- you can change this to another currency if you want

# Filter the DataFrame:
# - keep only rows with the chosen currency
# - keep only positive loan_amount values
mask = (df["currency"] == chosen_currency) & (df["loan_amount"] > 0)

df_currency = df.loc[mask].copy()

print(f"Number of loans in currency {chosen_currency}:", len(df_currency))
display(df_currency[["sector", "loan_amount"]].head())


Number of loans in currency USD: 105494


,sector,loan_amount
49,Retail,450.0
60,Agriculture,800.0
64,Housing,1500.0
70,Retail,2000.0
81,Housing,2000.0


In [7]:
# Group by sector and compute summary statistics
sector_summary = (
    df_currency
    .groupby("sector", as_index=False)
    .agg(
        loan_count=("id", "count"),
        avg_loan_amount=("loan_amount", "mean"),
        median_loan_amount=("loan_amount", "median")
    )
)

print(f"Number of sectors in currency {chosen_currency}:", len(sector_summary))
display(sector_summary.head())


Number of sectors in currency USD: 15


,sector,loan_count,avg_loan_amount,median_loan_amount
0,Agriculture,27844,1081.500503,800.0
1,Arts,2179,1821.581000,1000.0
2,Clothing,7147,1568.798097,1000.0
3,Construction,1113,2054.919137,1325.0
4,Education,8607,1390.809806,1000.0


In [8]:
# Sort sectors by average loan amount (largest first)
sector_by_avg_amount = sector_summary.sort_values(
    by="avg_loan_amount",
    ascending=False
)

print(f"Top 10 sectors by average loan amount in {chosen_currency}:")
display(sector_by_avg_amount.head(10))


Top 10 sectors by average loan amount in USD:


,sector,loan_count,avg_loan_amount,median_loan_amount
14,Wholesale,113,3272.345133,1500.0
5,Entertainment,280,3270.178571,2500.0
12,Services,8273,2239.659132,1500.0
3,Construction,1113,2054.919137,1325.0
7,Health,1613,1839.429634,1500.0
1,Arts,2179,1821.581000,1000.0
13,Transportation,1676,1693.899165,1200.0
2,Clothing,7147,1568.798097,1000.0
6,Food,17404,1490.089922,900.0
11,Retail,15094,1395.600901,1000.0


(Additional nugget) The following sectors have the most loans in the chosen currency:


In [9]:
# Sort sectors by number of loans (most common sectors)
sector_by_count = sector_summary.sort_values(
    by="loan_count",
    ascending=False
)

print(f"Top 10 sectors by number of loans in {chosen_currency}:")
display(sector_by_count.head(10))


Top 10 sectors by number of loans in USD:


,sector,loan_count,avg_loan_amount,median_loan_amount
0,Agriculture,27844,1081.500503,800.0
6,Food,17404,1490.089922,900.0
11,Retail,15094,1395.600901,1000.0
8,Housing,8802,874.792661,600.0
4,Education,8607,1390.809806,1000.0
12,Services,8273,2239.659132,1500.0
2,Clothing,7147,1568.798097,1000.0
10,Personal Use,4189,1141.250895,1000.0
1,Arts,2179,1821.581000,1000.0
13,Transportation,1676,1693.899165,1200.0


**Summary for Question 2**

In this question I filtered the dataset to keep only loans in one currency (`chosen_currency`).
Then I grouped the data by sector and calculated, for each sector, the number of loans and the average (and median) loan amount.
By sorting the results, I could see which sectors have the highest average loan sizes and which sectors are the most common in that currency.


## Question 3 – Do female-only, male-only, and mixed-gender groups receive different loan sizes?

The dataset contains a column with the genders of the borrowers in each loan.  
The text can contain multiple values (e.g. "female, female", "male, female", etc.).

In this section I will:
- Create a new simplified category for each loan based on `borrower_genders`, for example:
  - `female_only`
  - `male_only`
  - `mixed`
  - `unknown` (for missing or unclear values)
- Use `apply` with a small `lambda` function to generate this new column.
- Group the data by this new borrower type and calculate:
  - The average loan amount.
  - The average number of lenders (`lender_count`).
- Sort the results to see which groups receive larger loans and/or attract more lenders.

This helps explore possible differences between different types of borrower groups.


In [10]:
# Quick look at the borrower_genders column
print("Example values in borrower_genders:")
display(df["borrower_genders"].dropna().head(10))


Example values in borrower_genders:


,borrower_genders
0,female
1,"female, female"
2,female
3,female
4,female
5,female
6,female
7,female
8,female
9,female


In [11]:
import numpy as np

def classify_borrower_group(genders):
    """
    Takes the borrower_genders string and returns a simple category:
    - 'female_only'
    - 'male_only'
    - 'mixed'
    - 'unknown'
    """
    if pd.isna(genders):
        return "unknown"

    # Split by comma and clean whitespace
    parts = [g.strip().lower() for g in genders.split(",") if g.strip() != ""]

    if len(parts) == 0:
        return "unknown"

    unique_genders = set(parts)

    if unique_genders == {"female"}:
        return "female_only"
    elif unique_genders == {"male"}:
        return "male_only"
    elif unique_genders.issubset({"male", "female"}):
        return "mixed"
    else:
        # Just in case there are other unexpected values
        return "unknown"

# Use apply + lambda to create the new column
df["borrower_type"] = df["borrower_genders"].apply(lambda x: classify_borrower_group(x))

# Check the distribution of the new category
borrower_type_counts = df["borrower_type"].value_counts().reset_index()
borrower_type_counts.columns = ["borrower_type", "loan_count"]

print("Borrower type counts:")
display(borrower_type_counts)


Borrower type counts:


,borrower_type,loan_count
0,female_only,488081
1,male_only,138523
2,mixed,40380
3,unknown,4221


In [12]:
# Group by borrower_type and calculate summary stats
borrower_type_summary = (
    df
    .groupby("borrower_type", as_index=False)
    .agg(
        loan_count=("id", "count"),
        avg_loan_amount=("loan_amount", "mean"),
        median_loan_amount=("loan_amount", "median"),
        avg_lender_count=("lender_count", "mean")
    )
)

# Sort by average loan amount (largest first)
borrower_type_summary_sorted = borrower_type_summary.sort_values(
    by="avg_loan_amount",
    ascending=False
)

print("Borrower type summary (sorted by average loan amount):")
display(borrower_type_summary_sorted)


Borrower type summary (sorted by average loan amount):


,borrower_type,loan_count,avg_loan_amount,median_loan_amount,avg_lender_count
2,mixed,40380,1907.064141,1100.0,39.892595
3,unknown,4221,1173.655532,625.0,26.918977
1,male_only,138523,916.259394,625.0,22.512002
0,female_only,488081,730.487153,425.0,18.394103


In [13]:
borrower_type_by_lenders = borrower_type_summary.sort_values(
    by="avg_lender_count",
    ascending=False
)

print("Borrower type summary (sorted by average lender count):")
display(borrower_type_by_lenders)


Borrower type summary (sorted by average lender count):


,borrower_type,loan_count,avg_loan_amount,median_loan_amount,avg_lender_count
2,mixed,40380,1907.064141,1100.0,39.892595
3,unknown,4221,1173.655532,625.0,26.918977
1,male_only,138523,916.259394,625.0,22.512002
0,female_only,488081,730.487153,425.0,18.394103


**Summary for Question 3**

In this question I created a new categorical variable `borrower_type` from the `borrower_genders` column using `apply` and a `lambda` function.  
The categories are `female_only`, `male_only`, `mixed`, and `unknown`.  

Then I grouped the data by `borrower_type` and calculated, for each group, the number of loans, the average and median loan amount, and the average number of lenders.  
By comparing these values I can see whether female-only, male-only, or mixed groups tend to receive different loan sizes or attract different numbers of lenders.


## Question 4 – How do selected countries compare in loan amounts and lender counts?

In this question I want to focus on a small set of countries and compare them.  
I will manually choose a few countries (for example 3–5 countries that interest me).

I will:
- Use `.isin()` to filter the rows where the `country` is one of the selected countries.
- Optionally also filter to a single currency (for example `USD`) so that loan amounts are comparable.
- For each selected country, compute:
  - The average loan amount.
  - The average number of lenders.
- Sort the summary table by average loan amount or by average lender count.

This will show how these countries compare in terms of typical loan sizes and lender participation.


In [14]:
# Quick look at the most common countries
country_counts = df["country"].value_counts().reset_index()
country_counts.columns = ["country", "loan_count"]

print("Top 20 countries by number of loans:")
display(country_counts.head(20))


Top 20 countries by number of loans:


,country,loan_count
0,Philippines,160441
1,Kenya,75825
2,El Salvador,39875
3,Cambodia,34836
4,Pakistan,26857
5,Peru,22233
6,Colombia,21995
7,Uganda,20601
8,Tajikistan,19580
9,Ecuador,13521


In [15]:
# Choose a few countries to compare (you can change this list)
selected_countries = ["Philippines", "Kenya", "Peru", "Cambodia", "Pakistan"]

print("Selected countries:", selected_countries)

# Choose one currency to keep loan amounts comparable
chosen_currency_q4 = "USD"  # change if you want to focus on another currency
print("Chosen currency for Question 4:", chosen_currency_q4)


Selected countries: ['Philippines', 'Kenya', 'Peru', 'Cambodia', 'Pakistan']
Chosen currency for Question 4: USD


In [16]:
# Build masks for filtering
mask_countries = df["country"].isin(selected_countries)
mask_currency = df["currency"] == chosen_currency_q4
mask_amount = df["loan_amount"] > 0

# Apply all three conditions
df_countries = df.loc[mask_countries & mask_currency & mask_amount].copy()

print("Number of loans after filtering:", len(df_countries))
display(df_countries[["country", "currency", "loan_amount", "lender_count"]].head())


Number of loans after filtering: 6686


,country,currency,loan_amount,lender_count
60,Cambodia,USD,800.0,32
64,Cambodia,USD,1500.0,26
92,Cambodia,USD,1500.0,53
109,Cambodia,USD,600.0,22
127,Cambodia,USD,1000.0,31


In [17]:
# Group by country and compute summary statistics
country_summary = (
    df_countries
    .groupby("country", as_index=False)
    .agg(
        loan_count=("id", "count"),
        avg_loan_amount=("loan_amount", "mean"),
        median_loan_amount=("loan_amount", "median"),
        avg_lender_count=("lender_count", "mean")
    )
)

print("Country summary (unsorted):")
display(country_summary)


Country summary (unsorted):


,country,loan_count,avg_loan_amount,median_loan_amount,avg_lender_count
0,Cambodia,5338,1008.266205,1000.0,30.698389
1,Kenya,514,2548.297665,1500.0,75.258755
2,Pakistan,1,50000.000000,50000.0,805.000000
3,Peru,832,2311.748798,1400.0,53.473558
4,Philippines,1,25000.000000,25000.0,747.000000


In [18]:
# Sort by average loan amount (largest loans first)
country_by_avg_amount = country_summary.sort_values(
    by="avg_loan_amount",
    ascending=False
)

print("Selected countries sorted by average loan amount:")
display(country_by_avg_amount)

# Sort by average lender count (countries with more lenders first)
country_by_lenders = country_summary.sort_values(
    by="avg_lender_count",
    ascending=False
)

print("Selected countries sorted by average number of lenders:")
display(country_by_lenders)


Selected countries sorted by average loan amount:


,country,loan_count,avg_loan_amount,median_loan_amount,avg_lender_count
2,Pakistan,1,50000.000000,50000.0,805.000000
4,Philippines,1,25000.000000,25000.0,747.000000
1,Kenya,514,2548.297665,1500.0,75.258755
3,Peru,832,2311.748798,1400.0,53.473558
0,Cambodia,5338,1008.266205,1000.0,30.698389


Selected countries sorted by average number of lenders:


,country,loan_count,avg_loan_amount,median_loan_amount,avg_lender_count
2,Pakistan,1,50000.000000,50000.0,805.000000
4,Philippines,1,25000.000000,25000.0,747.000000
1,Kenya,514,2548.297665,1500.0,75.258755
3,Peru,832,2311.748798,1400.0,53.473558
0,Cambodia,5338,1008.266205,1000.0,30.698389


**Summary for Question 4**

In this question I selected a small set of countries using `.isin()` and restricted the data to a single currency so that loan amounts are comparable.  
For these countries I calculated, using `groupby`, the number of loans, the average and median loan amount, and the average number of lenders.

By sorting the results I could compare which of the selected countries tend to have larger loan sizes and which ones attract more lenders on average.


## Question 5 – Is there a relationship between the number of lenders and the loan amount?

Intuitively, we might expect that loans with more lenders are larger, but this is not obvious.

In this section I will:
- Focus on the `lender_count` and `loan_amount` columns.
- Optionally remove rows with zero or missing loan amounts.
- Create a new variable that groups loans into **lender count buckets**, for example:
  - `1–5 lenders`
  - `6–10 lenders`
  - `11–20 lenders`
  - `>20 lenders`
- Use `apply` with a `lambda` function to assign each loan to a bucket.
- Group by this new bucket and compute the average loan amount in each group.
- Sort the buckets to see if there is a clear trend.

This gives an idea of whether more lenders typically means a bigger loan.


In [19]:
# Quick summary of the lender_count column
print("Summary of lender_count:")
display(df["lender_count"].describe())


Summary of lender_count:


,lender_count
count,671205.000000
mean,20.590922
std,28.459551
min,0.000000
25%,7.000000
50%,13.000000
75%,24.000000
max,2986.000000


In [20]:
def classify_lender_bucket(n):
    """
    Takes the lender_count value and returns a bucket label:
    - 'unknown' for missing values
    - '1-5' for 1 to 5 lenders
    - '6-10' for 6 to 10 lenders
    - '11-20' for 11 to 20 lenders
    - '21-50' for 21 to 50 lenders
    - '51+' for more than 50 lenders
    """
    if pd.isna(n):
        return "unknown"

    if n <= 5:
        return "1-5"
    elif n <= 10:
        return "6-10"
    elif n <= 20:
        return "11-20"
    elif n <= 50:
        return "21-50"
    else:
        return "51+"

# Use apply + lambda to create the new bucket column
df["lender_bucket"] = df["lender_count"].apply(lambda x: classify_lender_bucket(x))

# Check the distribution of the new buckets
lender_bucket_counts = df["lender_bucket"].value_counts().reset_index()
lender_bucket_counts.columns = ["lender_bucket", "loan_count"]

print("Lender bucket counts:")
display(lender_bucket_counts)


Lender bucket counts:


,lender_bucket,loan_count
0,11-20,186025
1,6-10,156755
2,21-50,153535
3,1-5,122713
4,51+,52177


In [21]:
# Group by lender_bucket and compute summary statistics
lender_bucket_summary = (
    df
    .groupby("lender_bucket", as_index=False)
    .agg(
        loan_count=("id", "count"),
        avg_loan_amount=("loan_amount", "mean"),
        median_loan_amount=("loan_amount", "median"),
        avg_lender_count=("lender_count", "mean")
    )
)

print("Lender bucket summary (unsorted):")
display(lender_bucket_summary)


Lender bucket summary (unsorted):


,lender_bucket,loan_count,avg_loan_amount,median_loan_amount,avg_lender_count
0,1-5,122713,399.936437,200.0,2.978006
1,11-20,186025,597.961564,500.0,14.864018
2,21-50,153535,1157.685218,1000.0,31.321998
3,51+,52177,3237.246775,2575.0,88.729536
4,6-10,156755,372.893528,250.0,7.984096


In [22]:
# Define a custom order for the buckets
bucket_order_mapping = {
    "unknown": 0,
    "1-5": 1,
    "6-10": 2,
    "11-20": 3,
    "21-50": 4,
    "51+": 5
}

lender_bucket_summary["bucket_order"] = lender_bucket_summary["lender_bucket"].map(bucket_order_mapping)

# Sort by our custom order
lender_bucket_summary_sorted = lender_bucket_summary.sort_values("bucket_order")

print("Lender bucket summary (sorted by bucket order):")
display(lender_bucket_summary_sorted)


Lender bucket summary (sorted by bucket order):


,lender_bucket,loan_count,avg_loan_amount,median_loan_amount,avg_lender_count,bucket_order
0,1-5,122713,399.936437,200.0,2.978006,1
4,6-10,156755,372.893528,250.0,7.984096,2
1,11-20,186025,597.961564,500.0,14.864018,3
2,21-50,153535,1157.685218,1000.0,31.321998,4
3,51+,52177,3237.246775,2575.0,88.729536,5


In [24]:
lender_bucket_by_amount = lender_bucket_summary.sort_values(
    by="avg_loan_amount",
    ascending=False
)

print("Lender buckets sorted by average loan amount (largest first):")
display(lender_bucket_by_amount)


Lender buckets sorted by average loan amount (largest first):


,lender_bucket,loan_count,avg_loan_amount,median_loan_amount,avg_lender_count,bucket_order
3,51+,52177,3237.246775,2575.0,88.729536,5
2,21-50,153535,1157.685218,1000.0,31.321998,4
1,11-20,186025,597.961564,500.0,14.864018,3
0,1-5,122713,399.936437,200.0,2.978006,1
4,6-10,156755,372.893528,250.0,7.984096,2


**Summary for Question 5**

In this question I analysed the relationship between the number of lenders and the loan amount.  
First, I created a new categorical variable `lender_bucket` from the numeric `lender_count` column using `apply` and a `lambda` function. The buckets group loans into ranges such as `1–5`, `6–10`, `11–20`, `21–50`, and `51+` lenders.

Then I grouped the data by `lender_bucket` and calculated, for each bucket, the number of loans, the average and median loan amount, and the average number of lenders. By sorting the buckets, I could see how typical loan sizes change as the number of lenders increases.


## Question 6 – Do shorter-term loans get funded faster than longer-term loans?

The dataset contains timestamps for when loans were posted and when they were funded.

In this question I will:
- Convert the `posted_time` and `funded_time` columns to datetime format.
- Create a new column, for example `time_to_fund_days`, which measures how many days passed between posting and funding.
- Use either:
  - The raw `term_in_months` value, or
  - A new variable that groups loans into term buckets (e.g. `short` / `medium` / `long` term).
- Group the loans by term (or term bucket) and compute the average `time_to_fund_days` for each group.
- Sort the results to see if shorter-term loans are funded more quickly.

This analysis explores a possible relationship between loan duration and how fast lenders fund it.


In [25]:
# Convert posted_time and funded_time to datetime
df["posted_time"] = pd.to_datetime(df["posted_time"], errors="coerce")
df["funded_time"] = pd.to_datetime(df["funded_time"], errors="coerce")

print("Sample of posted_time and funded_time after conversion:")
display(df[["posted_time", "funded_time"]].head())


Sample of posted_time and funded_time after conversion:


,posted_time,funded_time
0,2014-01-01 06:12:39+00:00,2014-01-02 10:06:32+00:00
1,2014-01-01 06:51:08+00:00,2014-01-02 09:17:23+00:00
2,2014-01-01 09:58:07+00:00,2014-01-01 16:01:36+00:00
3,2014-01-01 08:03:11+00:00,2014-01-01 13:00:00+00:00
4,2014-01-01 11:53:19+00:00,2014-01-01 19:18:51+00:00


In [26]:
# Keep only rows where both timestamps are available
mask_times = df["posted_time"].notna() & df["funded_time"].notna()

df_time = df.loc[mask_times].copy()

# Compute the time difference in days
df_time["time_to_fund_days"] = (
    (df_time["funded_time"] - df_time["posted_time"]).dt.total_seconds() / (24 * 60 * 60)
)

# Optional: remove negative or extreme values, if any
df_time = df_time[df_time["time_to_fund_days"] >= 0]

print("Summary of time_to_fund_days:")
display(df_time["time_to_fund_days"].describe())

print("First few rows with time_to_fund_days:")
display(df_time[["posted_time", "funded_time", "time_to_fund_days"]].head())


Summary of time_to_fund_days:


,time_to_fund_days
count,622873.000000
mean,14.641579
std,14.399061
min,0.001389
25%,5.174282
50%,9.590127
75%,22.538704
max,420.573264


First few rows with time_to_fund_days:


,posted_time,funded_time,time_to_fund_days
0,2014-01-01 06:12:39+00:00,2014-01-02 10:06:32+00:00,1.162419
1,2014-01-01 06:51:08+00:00,2014-01-02 09:17:23+00:00,1.101562
2,2014-01-01 09:58:07+00:00,2014-01-01 16:01:36+00:00,0.252419
3,2014-01-01 08:03:11+00:00,2014-01-01 13:00:00+00:00,0.206123
4,2014-01-01 11:53:19+00:00,2014-01-01 19:18:51+00:00,0.309398


In [27]:
def classify_term_bucket(months):
    """
    Takes term_in_months and returns a bucket:
    - 'unknown' for missing values
    - 'short (<= 6m)' for terms up to 6 months
    - 'medium (7-12m)' for terms between 7 and 12 months
    - 'long (> 12m)' for terms longer than 12 months
    """
    if pd.isna(months):
        return "unknown"

    m = float(months)

    if m <= 6:
        return "short (<= 6m)"
    elif m <= 12:
        return "medium (7-12m)"
    else:
        return "long (> 12m)"

# Apply the function to create a new column in df_time
df_time["term_bucket"] = df_time["term_in_months"].apply(lambda x: classify_term_bucket(x))

print("Term bucket counts:")
display(df_time["term_bucket"].value_counts().reset_index())


Term bucket counts:


,term_bucket,count
0,long (> 12m),310792
1,medium (7-12m),269757
2,short (<= 6m),42324


In [29]:
# Group by the exact term_in_months
term_summary = (
    df_time
    .groupby("term_in_months", as_index=False)
    .agg(
        loan_count=("id", "count"),
        avg_time_to_fund=("time_to_fund_days", "mean"),
        median_time_to_fund=("time_to_fund_days", "median")
    )
)

# Sort by term_in_months
term_summary_sorted = term_summary.sort_values("term_in_months")

print("Summary by exact term_in_months:")
display(term_summary_sorted.head(20))


Summary by exact term_in_months:


,term_in_months,loan_count,avg_time_to_fund,median_time_to_fund
0,1.0,31,18.197301,14.870093
1,2.0,537,16.917812,9.225752
2,3.0,4871,61.538051,34.057199
3,4.0,1345,8.855381,5.973600
4,5.0,9610,6.613480,4.953877
5,6.0,25930,8.974365,6.070017
6,7.0,32808,6.464643,5.273096
7,8.0,142098,8.975647,6.676910
8,9.0,14370,16.131139,12.003484
9,10.0,18414,15.367555,11.349091


In [30]:
# Group by the term_bucket to compare categories
term_bucket_summary = (
    df_time
    .groupby("term_bucket", as_index=False)
    .agg(
        loan_count=("id", "count"),
        avg_time_to_fund=("time_to_fund_days", "mean"),
        median_time_to_fund=("time_to_fund_days", "median"),
        avg_term_months=("term_in_months", "mean")
    )
)

print("Term bucket summary (unsorted):")
display(term_bucket_summary)


Term bucket summary (unsorted):


,term_bucket,loan_count,avg_time_to_fund,median_time_to_fund,avg_term_months
0,long (> 12m),310792,17.746666,14.046939,18.436482
1,medium (7-12m),269757,11.072004,7.581979,8.846254
2,short (<= 6m),42324,14.591536,6.273275,5.309706


In [ ]:
# Define a custom order for the term buckets
term_bucket_order = {
    "short (<= 6m)": 1,
    "medium (7-12m)": 2,
    "long (> 12m)": 3,
    "unknown": 4
}

term_bucket_summary["bucket_order"] = term_bucket_summary["term_bucket"].map(term_bucket_order)

term_bucket_summary_sorted = term_bucket_summary.sort_values("bucket_order")

print("Term bucket summary (sorted by bucket order):")
display(term_bucket_summary_sorted)


**Summary for Question 6**

In this question I studied whether shorter-term loans are funded faster than longer-term loans.  
First, I converted the `posted_time` and `funded_time` columns to datetime format and created a new variable `time_to_fund_days`, which measures how many days it took for each loan to be fully funded.

Then I created a categorical variable `term_bucket` from `term_in_months` using `apply` and a `lambda` function, with buckets such as `short (<= 6m)`, `medium (7-12m)`, and `long (> 12m)`.  
By grouping the data by `term_bucket` and computing the average and median `time_to_fund_days`, I could compare how quickly different types of loans are funded.


## Question 7 – How many borrowers are in each loan, and does this vary by sector or country?

Some loans are for a single borrower, while others are for a group.  
We can estimate the number of borrowers from the `borrower_genders` column, where multiple genders are separated by commas.

In this section I will:
- Use `apply` with a `lambda` function on `borrower_genders` to:
  - Count how many genders are listed for each loan.
  - Store this in a new column such as `num_borrowers`.
- Handle missing values safely (for example, treat them as `num_borrowers = 0` or `1` depending on how I decide).
- Group by `sector` (and/or by `country`) and compute the average number of borrowers per loan.
- Sort the results by the average `num_borrowers` to see where group loans are more common.

This shows how group vs individual loans are distributed across sectors or countries.


In [ ]:
import numpy as np

def count_borrowers(genders):
    """
    Takes the borrower_genders string and returns the number of borrowers.
    - If the value is missing (NaN) or an empty string, return NaN.
    - Otherwise, split by comma and count the non-empty parts.
    """
    if pd.isna(genders):
        return np.nan

    parts = [g.strip() for g in genders.split(",") if g.strip() != ""]

    if len(parts) == 0:
        return np.nan

    return len(parts)

# Create the new column using apply + lambda
df["num_borrowers"] = df["borrower_genders"].apply(lambda x: count_borrowers(x))

print("Summary of num_borrowers:")
display(df["num_borrowers"].describe())

print("First few rows with borrower_genders and num_borrowers:")
display(df[["borrower_genders", "num_borrowers"]].head(10))


In [ ]:
# Group by sector and compute average number of borrowers
sector_borrowers_summary = (
    df
    .groupby("sector", as_index=False)
    .agg(
        loan_count=("id", "count"),
        avg_num_borrowers=("num_borrowers", "mean"),
        median_num_borrowers=("num_borrowers", "median")
    )
)

print("Sector-level summary of number of borrowers (unsorted):")
display(sector_borrowers_summary.head())


In [ ]:
# Sort sectors by average number of borrowers (largest first)
sector_by_borrowers = sector_borrowers_summary.sort_values(
    by="avg_num_borrowers",
    ascending=False
)

print("Top sectors by average number of borrowers per loan:")
display(sector_by_borrowers.head(10))

print("Sectors with the smallest average number of borrowers per loan:")
display(sector_by_borrowers.tail(10))


In [ ]:
# Group by country and compute average number of borrowers
country_borrowers_summary = (
    df
    .groupby("country", as_index=False)
    .agg(
        loan_count=("id", "count"),
        avg_num_borrowers=("num_borrowers", "mean"),
        median_num_borrowers=("num_borrowers", "median")
    )
)

# Keep only countries with a reasonable number of loans (e.g. at least 100)
country_borrowers_filtered = country_borrowers_summary[country_borrowers_summary["loan_count"] >= 100]

# Sort by average number of borrowers
country_by_borrowers = country_borrowers_filtered.sort_values(
    by="avg_num_borrowers",
    ascending=False
)

print("Countries (with at least 100 loans) sorted by average number of borrowers per loan:")
display(country_by_borrowers.head(15))


**Summary for Question 7**

In this question I estimated how many borrowers are involved in each loan by creating a new numeric column `num_borrowers` from the `borrower_genders` text. I used `apply` and a `lambda` function to split the gender string by commas and count how many non-empty entries there are.

Then I grouped the data by sector and calculated, for each sector, the number of loans and the average (and median) number of borrowers per loan. By sorting the results, I could see which sectors tend to have more group loans on average. I optionally repeated a similar analysis by country, focusing on countries with at least a certain number of loans.


## Question 8 – Are loans with certain tags different in size from the overall average?

The dataset includes a `tags` column with extra information about the purpose or characteristics of the loan.  
These tags are stored as text, often with several tags in the same field.

In this question I will:
- Explore which tags appear most frequently (at least in a simple way).
- Pick one or two tags of interest (for example containing words like "education" or "water").
- Use string methods (such as `.str.contains(...)`) to filter loans that include a given tag.
- Compare the average loan amount for loans with that tag against:
  - the overall average loan amount, or
  - loans without that tag.
- Optionally repeat for more than one tag.

This gives a simple view of whether certain tagged projects tend to have larger or smaller loan amounts.


In [ ]:
# Quick look at the tags column
print("Number of non-missing tags:", df["tags"].notna().sum())

print("\nExample values in tags:")
display(df["tags"].dropna().head(10))


In [ ]:
# Choose a keyword to search for in the tags
tag_keyword = "education"  # change this to try other keywords, e.g. "water", "health", "agriculture"
print("Chosen tag keyword:", tag_keyword)


In [ ]:
# Boolean masks for loans with and without the chosen tag keyword
mask_with_tag = df["tags"].str.contains(tag_keyword, case=False, na=False)
mask_without_tag = ~mask_with_tag

df_with_tag = df.loc[mask_with_tag].copy()
df_without_tag = df.loc[mask_without_tag].copy()

print(f"Number of loans WITH tag containing '{tag_keyword}':", len(df_with_tag))
print(f"Number of loans WITHOUT tag containing '{tag_keyword}':", len(df_without_tag))

# Compute average loan amount in both groups
avg_with_tag = df_with_tag["loan_amount"].mean()
avg_without_tag = df_without_tag["loan_amount"].mean()

print(f"\nAverage loan amount WITH '{tag_keyword}' tag: {avg_with_tag:.2f}")
print(f"Average loan amount WITHOUT '{tag_keyword}' tag: {avg_without_tag:.2f}")


In [ ]:
tag_keywords = ["education", "water", "health", "agriculture"]  # you can change this list

summary_rows = []

for kw in tag_keywords:
    mask_kw = df["tags"].str.contains(kw, case=False, na=False)

    df_kw = df.loc[mask_kw]
    df_not_kw = df.loc[~mask_kw]

    avg_kw = df_kw["loan_amount"].mean()
    avg_not_kw = df_not_kw["loan_amount"].mean()

    summary_rows.append({
        "tag_keyword": kw,
        "loan_count_with_tag": len(df_kw),
        "avg_loan_with_tag": avg_kw,
        "avg_loan_without_tag": avg_not_kw
    })

tag_summary = pd.DataFrame(summary_rows)

print("Comparison of average loan amounts for different tag keywords:")
display(tag_summary)


In [ ]:
tag_keywords = ["education", "water", "health", "agriculture"]  # you can change this list

summary_rows = []

for kw in tag_keywords:
    mask_kw = df["tags"].str.contains(kw, case=False, na=False)

    df_kw = df.loc[mask_kw]
    df_not_kw = df.loc[~mask_kw]

    avg_kw = df_kw["loan_amount"].mean()
    avg_not_kw = df_not_kw["loan_amount"].mean()

    summary_rows.append({
        "tag_keyword": kw,
        "loan_count_with_tag": len(df_kw),
        "avg_loan_with_tag": avg_kw,
        "avg_loan_without_tag": avg_not_kw
    })

tag_summary = pd.DataFrame(summary_rows)

print("Comparison of average loan amounts for different tag keywords:")
display(tag_summary)


**Summary for Question 8**

In this question I explored the `tags` column using string methods.  
I chose a keyword (for example `"education"`) and used `.str.contains()` to filter loans whose tags contain that keyword. Then I compared the number of loans and the average loan amount for loans with the keyword versus loans without it.

I also repeated this for a small list of tag keywords to see how different types of tagged projects compare. This gave a simple view of whether loans with certain tags tend to be larger or smaller than the overall average.


## Conclusion

In this notebook I explored the Kiva loans dataset using pandas and answered eight different questions.  
Across these questions I used:

- Row selection with string methods and numerical conditions.
- Sorting tables by one or more columns.
- Grouping and aggregating data with `groupby`.
- Creating new variables with `apply` and `lambda`.

The analyses gave some insights into sectors, countries, borrower types, loan sizes, and funding behaviour on Kiva.  
In the accompanying video I will briefly present six of these questions and highlight the most relevant pieces of code and results.
